# Import and Settings

In [5]:
import numpy as np
import random
from scipy.linalg import eig

## Settings and parameter

In [6]:
directed_graph = [[0,1,1,1],[0,0,1,1],[1,0,0,0],[1,0,1,0]]
m = 0.15

## Generate random directed graph

In [13]:
n = 20

# generates a graph from a given number of n
def generate_random_graph(number_of_nodes):
    generated_graph = []
    for i in range(number_of_nodes):
        generated_row = []
        for j in range(number_of_nodes):
            generated_row.append(random.randint(0,1))
        generated_graph.append(generated_row)
    return generated_graph

generate_random_graph(n)

[[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
 [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,

# Processing

## Dangling Nodes

In [63]:
def check_for_negatives(graph):
    for row in graph:
        for value in row:
            if value < 0:
                return True
    return False

In [64]:
def handling_dangly_nodes(graph):
    if check_for_negatives(graph):
        return
    test_graph = []
    for row in graph:
        if np.sum(row) == 0:
            test_graph.append([1/len(row) for _ in row])
        else:
            test_graph.append(row)
    return test_graph

In [65]:
A = handling_dangly_nodes(directed_graph)
A

[[0, 1, 1, 1], [0, 0, 1, 1], [1, 0, 0, 0], [1, 0, 1, 0]]

## Converting Graph

In [66]:
# Assumes it is a square matrix
def convert_graph_to_link_matrix(graph):
    transformed_graph = []
    for row in graph:
        col = []
        sum_row = sum(row)
        for val in row:
            col.append(val / sum_row)
        transformed_graph.append(col)
    transformed_graph = np.array(transformed_graph)
    transformed_graph = transformed_graph.T
    return transformed_graph

In [67]:
A = convert_graph_to_link_matrix(A)
A

array([[0.        , 0.        , 1.        , 0.5       ],
       [0.33333333, 0.        , 0.        , 0.        ],
       [0.33333333, 0.5       , 0.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

## Dangling Links

In [68]:
def handling_dangly_links(graph, m):
    S = np.full((len(graph), len(graph)), 1/len(graph))
    return ((1 - m) * graph) + (m * S)

In [69]:
A = handling_dangly_links(A, m)
A

array([[0.0375    , 0.0375    , 0.8875    , 0.4625    ],
       [0.32083333, 0.0375    , 0.0375    , 0.0375    ],
       [0.32083333, 0.4625    , 0.0375    , 0.4625    ],
       [0.32083333, 0.4625    , 0.0375    , 0.0375    ]])

## Eigen Values and Vector

In [70]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = eig(A)

# Define a tolerance level for eigenvalue proximity (e.g., 0.01)
tolerance = 0.01

# Find eigenvectors corresponding to eigenvalues close to 1
eigenvectors_close_to_1 = []
for i in range(len(eigenvalues)):
    if np.isclose(eigenvalues[i], 1, atol=tolerance):
        eigenvectors_close_to_1.append(eigenvectors[:, i])

# Convert the list of eigenvectors to a NumPy array
eigenvectors_close_to_1 = np.array(eigenvectors_close_to_1)

eigenvectors_close_to_1 = eigenvectors_close_to_1[0]

# Normalize
eigenvectors_close_to_1 = [rank / sum(eigenvectors_close_to_1) for rank in eigenvectors_close_to_1]
eigenvectors_close_to_1

[(0.3681506770476028+0j),
 (0.14180935849682083+0j),
 (0.2879616285976069+0j),
 (0.20207833585796964+0j)]